In [1]:
import os
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

In [2]:
from langchain.document_loaders import PDFPlumberLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
import chromadb

In [16]:
def load_chunk_persist_pdf() -> Chroma:
    pdf_folder_path = "data/"
    documents = []
    for file in os.listdir(pdf_folder_path):
        if file.endswith('.pdf'):
            pdf_path = os.path.join(pdf_folder_path, file)
            loader = PDFPlumberLoader(pdf_path)
            documents.extend(loader.load())
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=300)
    chunked_documents = text_splitter.split_documents(documents)
    client = chromadb.Client()
    if client.list_collections():
        consent_collection = client.create_collection("consent_collection")
    else:
        print("Collection already exists")
    vectordb = Chroma.from_documents(
        documents=chunked_documents,
        embedding=HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2'),
        persist_directory="chroma_store/"
    )
    vectordb.persist()
    return vectordb

In [17]:
vectordb = load_chunk_persist_pdf()

Collection already exists


In [18]:
vectordb = Chroma(persist_directory="chroma_store/", embedding_function=HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2'))
print(vectordb._collection.count())

3236


In [21]:
matching_docs = vectordb.similarity_search("곰팡이가 발생하면 어떤 문제가 생겨?")
matching_docs

[Document(page_content='승승화화성성잉잉크크는는상상온온에에서서도도전전사사((転転写写))됨됨\n충충전전물물((완완충충재재))로로서서사사용용되되면면시시간간의의경경과과와와함함께께상상품품\n((가가방방이이나나구구두두등등))에에전전사사', metadata={'Author': 'LKH', 'CreationDate': "D:20151026170221+09'00'", 'Creator': 'Hancom PDF 1.3.0.460', 'ModDate': "D:20151026170221+09'00'", 'PDFVersion': '1.4', 'Producer': 'Hancom PDF 1.3.0.460', 'Title': 'Microsoft PowerPoint - 벽지반점_조사연구_내화건축자재협회_최종_20151026.pptx', 'file_path': 'data/Wallpaper Survey.pdf', 'page': 22, 'source': 'data/Wallpaper Survey.pdf', 'total_pages': 84}),
 Document(page_content='한 일이라고 판단된다. 또한 본 연구에서 상세히 다루지 않은 설계 및 계획단\n계에서의 하자발생에 대한 세부적이고 전문적인 연구가 추후 계속 되어야 할\n것으로 사료된다.\n- 51 -', metadata={'Author': 'Administrator', 'CreationDate': 'D:20050924160932Z', 'Creator': 'PScript5.dll Version 5.2', 'ModDate': "D:20050924164531+09'00'", 'Producer': 'COXPDF 3.0', 'Title': 'Microsoft Word - 001.doc', 'file_path': 'data/001 Feb .pdf', 'page': 59, 'source': 'data/001 Feb .pdf', 'total_pages': 64}),
 Document(page_content='강에 악영향을 주고

In [23]:
matching_docs = vectordb.similarity_search("도배지에 곰팡이가 생겼을 때 높은 습도가 원인인 경우, 습기 관리는 어떻게 해야 할까요?")
matching_docs

[Document(page_content='또한 높았다. 즉, 타일공사 하자 유형 중 관련 하자가 적으나 한 번 하자가 발생하면 처리 후에도 지속적으로 재발하며 보수\n자체도 어려운 하자로, 타일 자재 및 시공 자체의 문제보다 시공 부위 하중 집중 현상, 바탕 구조체의 이상 변위, 콘크리트 바\n탕 균열 등 설계 및 타 공종에 의한 영향이 하자의 주요 원인인 것으로 분석되었다. 이에 설계 시 구조체의 이상 변위와 시공\n부위 하중 집중 현상을 사전에 예방할 필요가 있으며, 시공 단계에서 타 공종 작업 시 변형 방지를 위한 조치 방안 실시, 하중\n에 대응할 수 있는 탄성 접착제 사용하고 시공 후 보양 철저 등의 하자 저감 대책이 요구된다.\n‘오염 및 이색(e)’은 타 유형에 비해 하자 발생빈도, 하자 보수 후 재발 가능성, 하자 보수의 어려움 정도 모두 낮았다. 줄눈\n건조 전 오염물질 혼입, 오염된 공구 또는 작업장, 불량 자재 사용 등의 원인으로 발생하는 하자로써, 시공 단계에서 작업장\n청결 유지, 고온다습한 조건에서 시공 지양, 자재검수 및 시공교육 등의 대책과 유지 관리 단계에서 공동주택 거주자의 주의\n를 필요로 하지만, 추가적인 자원 배분 및 집중 관리가 아닌 중장기적인 개선 계획이 요구되며, 하자 저감을 위한 과잉노력을\n지양하고, 해당 하자에 투입되는 노력을 개선이 시급한 다른 유형에 집중할 필요가 있는 것으로 분석되었다.\n6. 결 론\n본 연구는 타일공사의 하자 데이터를 바탕으로 정량적 지표를 산출한 후, 전문가 의견이 반영된 AHP 분석 결과를 통해 5\n개의 타일공사 하자 유형에 대한 정성적 지표의 가중치를 산정하여 하자 유형을 분석하였으며, 도출된 결론은 다음과 같다.\n첫째, 수집한 타일공사의 하자 데이터를 분류한 후, 정량적인 지표인 하자 발생빈도를 분석하였다. 타일공사의 하자는 주\n로 욕실에서 발생하며, ‘균열 및 파손’, ‘ 줄눈불량’, ‘구배불량’이 하자 발생빈도가 높은 것으로 나타났다.', metadata={'Creati

In [24]:
matching_docs = vectordb.similarity_search("새집증후군의 주요 원인은 무엇인가요?")
matching_docs

[Document(page_content='한 일이라고 판단된다. 또한 본 연구에서 상세히 다루지 않은 설계 및 계획단\n계에서의 하자발생에 대한 세부적이고 전문적인 연구가 추후 계속 되어야 할\n것으로 사료된다.\n- 51 -', metadata={'Author': 'Administrator', 'CreationDate': 'D:20050924160932Z', 'Creator': 'PScript5.dll Version 5.2', 'ModDate': "D:20050924164531+09'00'", 'Producer': 'COXPDF 3.0', 'Title': 'Microsoft Word - 001.doc', 'file_path': 'data/001 Feb .pdf', 'page': 59, 'source': 'data/001 Feb .pdf', 'total_pages': 64}),
 Document(page_content='수 있다. 이는 우리나라가 갖고 있는 사회적 여건을 그대로 반영하는 것으로서\n노인계층의 인구비율 증가는 주거문제에 있어서 노인계층의 수요에 대한 대응이\n중요하게 부각될 것으로 예상된다. 특히, 인구구조의 변동과 더불어 전통적인 가족구\n성의 해체로 인한 핵가족화 및 노인부양 인력의 감소, 노인들의 의식 변화에 따른\n독립적 생활선호 등으로 인해 노인 독신가구 및 부부가구가 증가하고 있다. 노인들의\n의식변화는 노후에 자녀와 동거하면서 경제적으로 의존한다는 생각으로부터 별거하\n면서 경제적으로 의존하지 않으려는 방향으로 변하고 있어 자녀들과 동거하는 노인\n의 비율이 크게 감소하고 있다. 이러한 추세는 앞으로 더욱 심화될 것으로 전망되며,\n노인 단독가구의 비율이 점차 증가하고 있다.40) 실제로 2010년 통계청 장래가구추계\n39) 추선경, 브랜드아파트의단위평면변화와특성에관한연구, 서울과학기술대학원석사학위, p.27, 2011.', metadata={'CreationDate': "D:20080820165432+09'00'", 'Creat

In [25]:
matching_docs = vectordb.similarity_search("도배지에 생긴 반점을 없애기 위해 가장 효과적인 방법은 무엇인가요?")
matching_docs

[Document(page_content='협잡물용해시험(아세톤용액사용) 협잡물용해시험\n석고보드원지협잡물\n현장에서채취한시료의협잡물이아세톤용액분무후즉시용해되고확산용출됨', metadata={'Author': 'LKH', 'CreationDate': "D:20151026170221+09'00'", 'Creator': 'Hancom PDF 1.3.0.460', 'ModDate': "D:20151026170221+09'00'", 'PDFVersion': '1.4', 'Producer': 'Hancom PDF 1.3.0.460', 'Title': 'Microsoft PowerPoint - 벽지반점_조사연구_내화건축자재협회_최종_20151026.pptx', 'file_path': 'data/Wallpaper Survey.pdf', 'page': 25, 'source': 'data/Wallpaper Survey.pdf', 'total_pages': 84}),
 Document(page_content='협잡물용해시험(아세톤용액사용) 협잡물용해시험\n석고보드원지협잡물\n현장에서채취한시료의협잡물이아세톤용액분무후즉시용해되고확산용출됨', metadata={'Author': 'LKH', 'CreationDate': "D:20151026170221+09'00'", 'Creator': 'Hancom PDF 1.3.0.460', 'ModDate': "D:20151026170221+09'00'", 'PDFVersion': '1.4', 'Producer': 'Hancom PDF 1.3.0.460', 'Title': 'Microsoft PowerPoint - 벽지반점_조사연구_내화건축자재협회_최종_20151026.pptx', 'file_path': 'data/Wallpaper Survey.pdf', 'page': 25, 'source': 'data/Wallpaper Survey.pdf', 'total_pages': 84}),
 Document(page_content='협잡물용해시험(아세톤용